<img src="https://automatetheboringstuff.com/images/000037.jpg">

In [77]:
#We'll need an interface for user input. Suggestion to create an empty xlsx template and to ask users to fill it.
#By default, the grid is converted to a table.
import pandas as pd
import numpy as np
dict_grid={
    'A':[5,6,'',8,4,7,'','',''],
    'B':[3,'',9,'','','',6,'',''],
    'C':['','',8,'','','','','',''],
    'D':['',1,'','',8,'','',4,''],
    'E':[7,9,'',6,'',2,'',1,8],
    'F':['',5,'','',3,'','',9,''],
    'G':['','','','','','',2,'',''],
    'H':['','',6,'','','',8,'',7],
    'I':['','','',3,1,6,'',5,9]
    
}
grid=pd.DataFrame(data=dict_grid)
grid

,A,B,C,D,E,F,G,H,I
0,5,3,,,7,,,,
1,6,,,1,9,5,,,
2,,9,8,,,,,6,
3,8,,,,6,,,,3
4,4,,,8,,3,,,1
5,7,,,,2,,,,6
6,,6,,,,,2,8,
7,,,,4,1,9,,,5
8,,,,,8,,,7,9


In [78]:
#define variables for the sub-grids:
#example:
upper_left=grid.iloc[ :3, :3]
#the subgrids will be numbered from 0 to 8, from top-left to bottom-right. upper_left corresponds to sub_grids[0]
sub_grids=[]
i=1
while i <10:
    col=0
    while col<9:
        if i in [1,2,3]:
            row=0
        elif i in [4,5,6]:
            row=3
        elif i in [7,8,9]:
            row=6
        sub=grid.iloc[row:row+3,col:col+3]
        sub_grids.append(sub)
        col+=3
        i+=1
sub_grids[8]

,G,H,I
6,2,8,
7,,,5
8,,7,9


### Algorithm:
- for each NaN, try a number from 1 to 9:
- check that the number doesn't exist on the line
- check that it doesn't exist on the column
- check that it doesn't exist on the 3x3 sub-grid
- in doubt, skip and go back to it at the end of the iteration

In [ ]:
#define a class 'cell' with parameters location, filled (boolean) and sub_grid
class cell():
    

In [9]:

digit=1
while digit <=9:
    for row in range(1,10):
        for cell in grid.loc[row]:
            print(list(grid.loc[row]))
            if digit not in list(grid.loc[row]):
                if digit not in list(list(grid[grid.columns[row]])):
                    if digit not in 

1
2
3
4
5
6
7
8
9


In [ ]:
#Harder level: in case of doubt define a restricted list of digits to choose from in each cell